# **Mission of clustering notebook**

You are a member of a bioinformatics team investigating the [effect of spaceflight on astronaut health](https://en.wikipedia.org/wiki/Effect_of_spaceflight_on_the_human_body).  Your team is trying to find the [biological pathways](https://en.wikipedia.org/wiki/Biological_pathway) that respond to exposure of adverse conditions in space such as [microgravity](https://www.nasa.gov/centers-and-facilities/glenn/what-is-microgravity/) and [radiation](https://www.nasa.gov/directorates/esdmd/hhp/space-radiation/).  Ultimately your team will need phenotype data as labels to train your supervised learning models, but for now, all you have is the gene expression data in the form of [RNA-seq](https://en.wikipedia.org/wiki/RNA-Seq).  

Your mission is to use unsupervised machine learning - [clustering](https://en.wikipedia.org/wiki/Cluster_analysis) - to determine if the astronaut samples and their ground control counterparts have clearly distinguishable gene expression profiles.  For if they do, then there is a chance that a supervised learning method can predict responses to spaceflight using different combinations and weights of gene expression from the RNA-seq data.

In this notebook, you will use k-means, PCA, and a heatmap to reach your conclusions.

# Read in the methods

Recall that we have put all the custom python methods in a separate notebook which you copied to your Google drive.  We need to read those methods into this notebook so that we can use them here.  You will get prompted to select the gmail address to use to permit access to your google drive for this notebook.

Note that we will import the methods in the notebook as "m", so all subsequent references to methods in that notebook will be prefixed with "m.".

In [1]:
# install and import the python module for importing a notebook
!pip install import_ipynb
import import_ipynb

In [2]:
# mount your google drive to this notebook
from google.colab import drive
drive.flush_and_unmount()
drive.mount("mnt", force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
# import the "Copy of methods.ipynb" from your google drive into this notebook (this will take a while -- mabye 5 minutes?)
m = __import__("mnt/MyDrive/Colab Notebooks/Copy of methods")

# read in data and metadata

The data that we will be using is [normalized](https://en.wikipedia.org/wiki/Normalization_(statistics)) RNA-seq data which was generated from retinal tissue.  The counts of genes per sample are represented in a table and have been normalized, which means they've been changed to account for differences in how the RNA-seq experiment played out.  For example, some transcripts may have been copied multiple times in the experiment (called [sequencing depth](https://www.biostars.org/p/282708/)) while others may not have been as deeply copied.  Moreover, because some genes are much longer than others, longer genes will have more transcript fragments and appear to be more highly expressed than shorter genes.  These differences will change the results of our analysis because we are looking for differences in gene expression (i.e. counts) between genes and between samples.
 Normalization is a statistical process of correcting for these differences, and there are many methods available.  The goal of normalization is to make the gene expression profiles more comparable across samples, allowing for accurate comparisons and statistical analysis.

In [ ]:
# read in the RNA-seq data from OSD-255
data=dict()
metadata=dict()
data['255-normalized'] = m.read_rnaseq_data('255_rna_seq_Normalized_Counts')
metadata['255'] = m.read_meta_data('255')

In [ ]:
# display the dimensions of the RNA-seq data set and the associated metadata
print('data shape: ', data['255-normalized'].shape)
print('metadata shape: ', metadata['255'].shape)

# Use the K-means algorithm to cluster the RNA-seq data

In [ ]:
# create a copy of the normalized RNA-seq data from OSD-255
df = data['255-normalized']

In [ ]:
# filter out nans and remove genes with CV < 1.5
print('before filter: ', df.shape)
df = m.filter_data(df, dropnans=True, dropprotein=None, droplowcvs=0)
print('after filter: ', df.shape)

In [ ]:
# run the my_kmeans method with k=2
m.my_kmeans(df, metadata, k=2)

In [ ]:
m.find_k_elbow(df)

In [ ]:
# now build kmeans with k=elbow
m.my_kmeans(df, metadata, k=3)

In [ ]:
# it seems like GSM3932697 / Retina_Flight_17 and GSM3932706 / Retina_Flight_18 are getting confused
# try removing them and rerunning WCCS
for sample in df.columns[1:]:
  print(sample)
  if sample == 'GSM3932697' or sample == 'GSM3932706':
    df = df.drop(columns=[sample], inplace=False)
m.my_kmeans(df, metadata, k=3)

It looks like the gene expression of all the ground control samples clusters uniquely into group 0.   This validates our suspicion that GSM3932697 and GSM3932706 are likely outliers, but it's difficult to tell by just looking at cluster membership. What we need is a visual way to view how the samples cluster and separate between ground control and spaceflight.  For that we turn to PCA clustering.

**QUESTIONS**

1. Using the histogram of CVs, about how many genes have a CV of 0.5 or less?  

2. After removing the 2 outliers GSM3932697 and GSM3932706 and re-running k-means, there seems to be one more outlier.  Which sample is it?

3. Given these results, would you say that spaceflight and ground control samples have distinct gene expression profiles?

**Double click here to enter your answers to the questions above.**

1.

2.

3.

# Use PCA to cluster and plot the RNA-seq data

Let's use the PCA clustering technique so we can visualize the data in 2-dimensions to see how spaceflight and ground control samples cluster and separate.

In [ ]:
# initialize df
df = data['255-normalized']


In [ ]:
# transpose the dataframe in preparation for PCA
print('shape before transpose: ', df.shape)
X = df.drop(columns=['Unnamed: 0']).to_numpy().T
print('shape after transpose: ', X.shape)


In [ ]:
# create an array y that represents whether the sample is ground control or spaceflight
y = m.np.array(list(metadata['255']['Factor Value[Spaceflight]']))
target_names=m.np.array(['Ground Control', 'Space Flight'])

# run PCA to reduce dimensions from 23,419 to 2!
pca = m.PCA(n_components=2)
X_r = pca.fit_transform(X)

# Percentage of variance explained for each components
print(
    "explained variance ratio (first two components): %s"
    % str(pca.explained_variance_ratio_)
)

# plot the pca plot
m.plt.figure()
colors = ["navy", "turquoise"]
lw = 2
for color, i, target_name in zip(colors, ['Ground Control', 'Space Flight'], target_names):
    m.plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=0.8, lw=lw, label=target_name)
m.plt.legend(loc="best", shadow=False, scatterpoints=1)
m.plt.title("PCA of OSD-255 dataset")

# show the last 3 digits of the sample name
samples = list(df.columns)[1:]
for i, txt in enumerate(samples):
    m.plt.annotate(txt[-3:], (X_r[i][0], X_r[i][1]))
m.plt.show()


We might choose to eliminate those outliers from both spaceflight and ground control for downstream analysis.  But we are already low on data, so removing data points is expensive.  And actually, we're not using the PCA plot for modeling the data and making predictions so there's no real concept of accuracy or performance.  Instead, we're using it to get a visual representation of the distribution of the data.  Now  we see how the ground control and space flight samples cluster together except for a few outliers.

**QUESTIONS**

1. Which samples are the outlier space flight samples?

2. Which samples are the outlier ground control samples?

3. Do these PCA clusters corroborate what we found using K-means in terms of data separation and outliers?

**Double click here to enter your answers to the questions above.**

1.

2.

3.

# Use the DESeq2 implementation of PCA

[DESeq2](https://www.bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html) is a software implementation that performs [differential gene expression analysis](https://en.wikipedia.org/wiki/Gene_expression_profiling) to identify which genes are differentially expressed (highly up-regulated or down-regulated) relative to other genes in the expression profile.  This tool is used to identify which genes may be responsible for a particular phenotype.  

In addition to performing differential gene expression analysis, DESeq2 also has an implementation of PCA.  This section of the notebook explores using it to compare against the `pydeseq2` implementation results.

In [ ]:
# read in data
df = data['255-normalized']

In [ ]:
dfT = df.T
dfT.columns=dfT.iloc[0]
dfT=dfT.iloc[1:]
dfT.columns.name=None
dfT = dfT.reset_index().rename(columns={"index":"sample"})


In [ ]:
dfT.shape

In [ ]:
condition_dict=dict()
for sample in list(dfT['sample']):
    val=metadata['255'][metadata['255']['Sample Name']==sample]['Factor Value[Spaceflight]'].values[0]
    if val == 'Ground Control':
        condition_dict[sample] = 0
    else:
        condition_dict[sample] = 1
dfT["condition"] = dfT["sample"].map(condition_dict)
conditions=dfT[['sample', 'condition']]

In [ ]:
counts=dfT.drop(columns=['sample', 'condition']).reset_index(drop=True)
counts.applymap(m.np.isreal)
counts=counts.astype(int)


In [ ]:
dds=m.DeseqDataSet(counts=counts, metadata=conditions, design_factors="condition")
dds.deseq2()

In [ ]:
m.sc.tl.pca(dds)
m.sc.pl.pca(dds, color='condition', size=200)

**QUESTIONS**

1. What is the shape of the dataframe used as input to count the gene expression?  

2. How and why is this different than the shape of the original RNA-seq dataframe?

3. Do the 2 PCA plots agree in their distribution in 2 dimensions?

**Double click here to enter your answers to the questions above.**

1.

2.

3.

# Use Gaussian mixture model to cluster the RNA-seq data


In [ ]:
# read in data
df = data['255-normalized']

In [ ]:
# filter data
print('shape before filter: ', df.shape)
df = m.filter_data(df, dropnans=False, dropprotein=None, droplowcvs=1.5)
print('shape after filter: ', df.shape)

In [ ]:
# build gmm with 3 distributions (this may take several minutes with lots of data)
gmm = m.my_gmm(df, metadata, 3)

**QUESTIONS**

1. How many genes were dropped from the dataframe after dropping 1.5 or lower CVs?  What do you think the effect will be on the quality of the clustering after dropping so many genes from the data set?

2. Does the GMM algorithm require that you specify the number of clusters up-front, or does the algorithm find the ideal number for you?

3. Do the GMM cluster results resemble those of k-means?  Why or why not?

**Double click here to enter your answers to the questions above.**

1.

2.

3.

# Plot the RNA-seq gene expression data in a heatmap

In [ ]:
# read in data
df = data['255-normalized']

# filter data
print('shape before filter: ', df.shape)
df = m.filter_data(df, dropnans=True, dropprotein='non-coding', droplowcvs=1.5)
print('shape after filter: ', df.shape)

# transpose the dataframe
dfT = df.T
dfT.columns=dfT.iloc[0]
dfT=dfT.iloc[1:]
dfT.columns.name=None
dfT = dfT.reset_index().rename(columns={"index":"sample"})


# assign a "condition" to each sample as either 1 (space flight) or 0 (ground control)
condition_dict=dict()
for sample in list(dfT['sample']):
    val=metadata['255'][metadata['255']['Sample Name']==sample]['Factor Value[Spaceflight]'].values[0]
    if val == 'Ground Control':
        condition_dict[sample] = 0
    else:
        condition_dict[sample] = 1
dfT["condition"] = dfT["sample"].map(condition_dict)
conditions=dfT[['sample', 'condition']]


# turn counts into integer values (normalized data is type float)
counts=dfT.drop(columns=['sample', 'condition']).reset_index(drop=True)
#counts.applymap(m.np.isreal)
counts=counts.astype(int)

# run a DESeq2 experiment that finds differential gene expression between the 2 conditions
# note we are not doing differential gene expression analysis but need to run this to get a PCA plot
dds=m.DeseqDataSet(counts=counts, metadata=conditions, design_factors="condition")
dds.deseq2()



In [ ]:
dds.layers['log1p'] = m.np.log1p(dds.layers['normed_counts'])

In [ ]:
grapher = m.pd.DataFrame(dds.layers['log1p'].T, index=dds.var_names, columns=list(conditions['sample']))
grapher.head()

In [ ]:
m.sns.clustermap(grapher, z_score=0, cmap='RdYlBu_r')

**QUESTIONS**

1.  On which axis do the 16 samples appear, vertical or horizontal?  the genes?

2.  Which 2 samples are least similar to sample GSM3932702?  How did you choose them?

3.  Which gene and sample have the hottest expression?  The coldest?

**Double click here to enter your answers to the questions above.**

1.

2.

3.